# Importing Important Libraries

In [3]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import gensim
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import ast

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Importing Dataset

In [4]:
movies_keyword_with_description_keyword = pd.read_csv("movies_keyword_with_description_keyword.csv")
movies_keyword_with_description_keyword = movies_keyword_with_description_keyword.drop(columns = ['Unnamed: 0.1'])

#Converting All keywords to Lowercases

In [5]:
num_rows = 75
num_columns = 498
for row_index in range(0,num_rows):
    for col_index in range(0,num_columns):
       value = movies_keyword_with_description_keyword.iloc[row_index, col_index]
       if isinstance(value, str):
          movies_keyword_with_description_keyword.iloc[row_index, col_index] = value.lower()

#Combining all keywords of each movie in single string(To Drop NA)

Creating a new dataframe which contain two columns:


*   Movie Name
*   All Keywords(Combined in single string)

This has been done in above Table (new_dataset) but in new_dataset we couldnot drop NA to drop NA we are creating new column in which all keywords are combined.


In [6]:
  #Creating dictonary to store movies name as keys and keywords as values
movies_combined_keywords = {}
for i in movies_keyword_with_description_keyword.columns:
   combined_keywords = []
   for j in movies_keyword_with_description_keyword[i]:
     combined_keywords.append(j)
     movies_combined_keywords.update({i:combined_keywords}) #updaing values in dictonary

Creating dataframe with dictonary

In [7]:
df_with_combined_keywords = pd.DataFrame(list(movies_combined_keywords.items()), columns=['Movie Titles', 'Movies Keywords'])
#df_with_combined_keywords.to_csv("df_with_combined_keywords.csv")

Combining all the keywords of each movie in single string (Excluding nulls)

In [8]:
df_with_combined_keywords['Movies Combined Keywords'] = df_with_combined_keywords['Movies Keywords'].apply(lambda x: ''.join(str([word for word in x if pd.notna(word)])))
#df_with_combined_keywords.to_csv("movies_combined_keywords_without_null.csv")

#Combining all keywords without null
Creating a column same as Movies Keywords but without null

In [9]:
df_with_combined_keywords['Movies New Keywords'] = df_with_combined_keywords['Movies Combined Keywords'].apply(ast.literal_eval)
df_with_combined_keywords
#df_with_combined_keywords.to_csv("movies_keyword_with_description_keyword_updated.csv")

,Movie Titles,Movies Keywords,Movies Combined Keywords,Movies New Keywords
0,The Shawshank Redemption,"[course, several, years, two, convicts, form, ...","['course', 'several', 'years', 'two', 'convict...","[course, several, years, two, convicts, form, ..."
1,The Godfather,"[vito, corleone, head, mafia, family, decides,...","['vito', 'corleone', 'head', 'mafia', 'family'...","[vito, corleone, head, mafia, family, decides,..."
2,The Dark Knight,"[menace, known, joker, wreaks, havoc, chaos, p...","['menace', 'known', 'joker', 'wreaks', 'havoc'...","[menace, known, joker, wreaks, havoc, chaos, p..."
3,The Godfather Part II,"[early, life, career, vito, corleone, 1920s, n...","['early', 'life', 'career', 'vito', 'corleone'...","[early, life, career, vito, corleone, 1920s, n..."
4,De 12 gezworenen,"[jury, new, york, city, murder, trial, frustra...","['jury', 'new', 'york', 'city', 'murder', 'tri...","[jury, new, york, city, murder, trial, frustra..."
...,...,...,...,...
493,F9: The Fast Saga,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['fast and furious franchise', 'car', 'gunfigh...","[fast and furious franchise, car, gunfight, sh..."
494,Harry Potter and the Sorcerer's Stone,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['magic', 'wizard', 'orphan', 'school of magic...","[magic, wizard, orphan, school of magic, based..."
495,The Marvelous Mrs. Maisel,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['female protagonist', 'new york city', 'jewis...","[female protagonist, new york city, jewish fam..."
496,Demon Slayer: Kimetsu no Yaiba,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['demon slayer', 'anime', 'brother sister rela...","[demon slayer, anime, brother sister relations..."


#Applying TF*IDF Vectorizer

Creating TF*IDF model

In [38]:
model = TfidfVectorizer()
X = model.fit_transform(df_with_combined_keywords['Movies Combined Keywords'])

#Applying Count Vectorizer (Alternative of TF*IDF)

In [34]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=498,stop_words='english')
vector = cv.fit_transform(df_with_combined_keywords['Movies Combined Keywords']).toarray()

#Applying Cosine Similarity

In [39]:
similarity = cosine_similarity(X)

#Lowering all the titles
Lowering all the titles of the movies so that searching could be easy

In [12]:
for index in range(len(df_with_combined_keywords["Movie Titles"])):
  df_with_combined_keywords["Movie Titles"][index] = df_with_combined_keywords["Movie Titles"][index].lower()

#Recommendation Function
Recommendation function that recommend similar movies based on movies title

In [46]:
def recommend(movie_name):
  movie_index = df_with_combined_keywords[df_with_combined_keywords["Movie Titles"]==movie_name].index[0]
  distances = similarity[movie_index]
  movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
  for i in movies_list:
    print(df_with_combined_keywords["Movie Titles"].iloc[i[0]])

#Getting the recommendation based on movies

In [47]:
recommend("the batman")

batman begins
the dark knight
spider-man: across the spider-verse
spider-man: into the spider-verse
the dark knight rises
